# 🚀 Korpusanalyse – Textkomplexität berechnen

## Hinweise zur Ausführung des Notebooks
Dieses Notebook kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../markdown/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen.
   
## Übersicht
Im Folgenden wird die Textkomplexität der einzelnen Pressemitteilungen mit der Python-Bibliothek [readability](https://pypi.org/project/readability/) errechnet, für unterschiedliche Zeitabschnitte (Monate, Jahre) zusammengefasst und visualisiert. 
1. Einlesen des Korpus und der Metadaten
2. Errechnen der Komplexitätsmaße

In [ ]:
# import libraries for table processing and for compuation of readability metrics
from pathlib import Path
from multiprocessing import Pool, cpu_count
from functools import partial
import requests

import pandas as pd
import textstat
textstat.set_lang("de")

## 1. Einlesen der Daten und Metadaten
Exemplarisch lesen wir einen Text ein und extrahieren die Metadaten des Texts aus der Metadaten-Tabelle, um in einem späteren Schritt das Datum der Veröffentlichung extrahieren zu können, um die Textkomplexität diachron zu analysieren.

### Text einlesen

In [ ]:
# 🚀 Create data directory path
corpus_path = Path(r"../data/txt")
if not corpus_path.exists():
    corpus_path.mkdir()

In [ ]:
# 🚀 Load the text file from GitHub 
! wget https://raw.githubusercontent.com/quadriga-dk/Text-Fallstudie-2/refs/heads/main/data/txt/2011-01-04_id8787.txt -P ../data/txt

In [ ]:
# set path to text
text_path = Path(r"../data/txt/1000035.txt")

# read text
text = text_path.read_text()
print(text)

In [ ]:
# 🚀 Load the metadata file from GitHub 
! wget https://raw.githubusercontent.com/quadriga-dk/Text-Fallstudie-2/refs/heads/main/data/metadata.csv -P ../data

### Metadaten einlesen

In [ ]:
# set path to metadata
metadata_path = "../data/metadata.csv"

# read metadata
metadata_df = pd.read_csv(metadata_path, sep=",")

Ersten fünf Zeilen anzeigen lassen:

In [ ]:
metadata_df.head()

Metadatum anhand von Dateinamen extrahieren:

In [ ]:
current_meta_data = metadata_df[metadata_df.filename == text_path.name]
current_meta_data

## 2. Textkomplexität berechnen 
Im folgenden berechnen wir mittels vier unterschiedlicher Metriken die Textkomplexität. Alle Metriken verwenden für die Berechnung des Komplexitätsscores die durchschnittliche Satzlänge und nehmen unterschiedliche Worteingenschaften dazu. 
Zuerst verschaffen wir uns einen Überblick über die Texteigenschaften, um die Ergebnisse der Metriken besser bewerten zu können. 

In [ ]:
print(f"Anzahl an Buchstaben: {textstat.letter_count(text)}")
print(f"Anzahl an Sätzen: {textstat.sentence_count(text)}")
print(f"Anzahl an Wörtern: {textstat.lexicon_count(text)}")
print(f"Durchschnittliche Anzahl an Buchstaben pro Wort: {textstat.avg_character_per_word(text)}")
print(f"Durchschnittliche Anzahl an Silben pro Wort: {textstat.avg_syllables_per_word(text)}")
print(f"Durchschnittliche Satzlänge: {textstat.avg_sentence_length(text)}")
print(f"Anzahl an Wörtern mit drei oder mehr Silben: {textstat.polysyllabcount(text)}")
print(f"Anzahl an langen Wörtern: {textstat.long_word_count(text)}")

### 2.1 Flesch 
Berechnung:
`180 - SL - (58,5 × ASW)` \
SL = Durchschnittliche Satzlänge (Anzahl der Wörter geteilt durch Anzahl der Sätze) \
ASW = Durchschnittliche Silbenzahl pro Wort (Anzahl der Silben geteilt durch Anzahl der Wörter) \

Interpretation: 0 (sehr schwierig) bis 100 (sehr leicht)




In [ ]:
textstat.flesch_reading_ease(text)

Ein Score zwischen 30 und 50 gibt eine hohe Schwierigkeit an, ungefährt äquivalent zum Niveau von Bachelorstudierenden.

### 2.2 Wiener Sachtextformel
Berechnung: `(0,2007 × MS) + (0,1682 × ASL) + (0,1373 × IW) - 2,779`

MS = Prozentsatz der Wörter mit drei oder mehr Silben \
SL = Durchschnittliche Satzlänge (Anzahl der Wörter geteilt durch Anzahl der Sätze) \
IW = Anzahl an langen Wörtern (länger als sechs Buchstaben) \

4 (leicht) bis 15 (schwierig), orientiert an Schulklassen


In [ ]:
textstat.wiener_sachtextformel(text, variant=2)

Die aufs Deutsche ausgelegte Wiener Sachtextformel stuft den Text auch als schwierig ein, etwa auf das Niveau der zwölften Klasse. 

### 2.3 Automated Readability Index (ARI)
Berechnung: 4,71 × WL + 0,5 × SL - 21,43

WL: Durchschnittliche Wortlänge in Buchstaben (Anzahl der Zeichen geteilt durch Anzahl der Wörter) \
SL: Durchschnittliche Satzlänge (Anzahl der Wörter geteilt durch Anzahl der Sätze) \

1 (sehr leicht) bis 14 (schwierig), orientiert an Schulklassen

In [ ]:
textstat.automated_readability_index(text)

Laut dem ARI wird der Text als äußerst schwieriger eingestuft und ist mit einem Score über 14 außerhalb der festgelegten Skala.

### 2.4 Coleman-Liau Index
Berechnung: `0,0588 × L - 0,296 × S - 15,8`



L = Durchschnittliche Anzahl von Buchstaben pro 100 Wörter \
S = Durchschnittliche Anzahl von Sätzen pro 100 Wörter \

Interpretation: 1 (sehr leicht) bis 14 (schwierig), orientiert an Schulklassen


In [ ]:
textstat.coleman_liau_index(text)

Wie bei ARI liegt auch beim Coleman-Liau Index der Index außerhalb der angedachten Skala und ist somit als sehr schwierig zu beurteilen.

## 3. Berechnung auf dem gesamten Korpus 

In [ ]:
# 🚀 Create download list 
github_api_txt_dir_path = "https://github.com/quadriga-dk/Text-Fallstudie-2/tree/main/data/txt"
txt_dir_info = requests.get(github_api_txt_dir_path).json()
url_list = [entry["download_url"] for entry in txt_dir_info]

# 🚀 Write download list as txt file
url_list_path = Path("github_txt_file_urls.txt")
with url_list_path.open('w') as output_txt:
    output_txt.write("\n".join(url_list))

In [ ]:
# ⚠️ Only execute, if you haven't downloaded the files yet!
# 🚀 Download all txt files – this step will take a while
! wget -i github_txt_file_urls.txt -P ../data/txt

Setzen des Pfads zum Korpus-Ordner:

In [ ]:
corpus_path = Path(r"../data/txt/")

Korpus einlesen:

In [ ]:
# Create dictionary to save the filenames and the text
corpus = {"filename":[], "text":[]}

# Iterate corpus directory, read text, save filename and text to corpus dict
for fp in corpus_path.iterdir():
    if fp.is_file():
        corpus["filename"].append(fp.name)
        corpus["text"].append(fp.read_text())

Scores pro Text berechnen:

In [ ]:
readability_scores = {}

# for speed, we use multiprocessing for computing the scores 
with Pool(cpu_count()) as p:
    readability_scores["Flesch"] = p.map(textstat.flesch_reading_ease, corpus["text"])
    readability_scores["Wiener_Sachtextformel"] = p.map(partial(textstat.wiener_sachtextformel, variant=1), corpus["text"])
    readability_scores["ARI"] = p.map(textstat.automated_readability_index, corpus["text"])
    readability_scores["Coleman_Liau"] = p.map(textstat.coleman_liau_index, corpus["text"])

# add scores to corpus dictionary
for measure, score in readability_scores.items():
    corpus[measure] = score

# transform to DataFrame
corpus_df = pd.DataFrame(corpus)
corpus_df.head()

## 4. Zusammenfügen der Scores mit den Metadaten 
Korpusdaten mit dem Metadaten zusammenfügen. Die Einträge werden über den Dateinamen einander zugeordnet.

In [ ]:
metadata_df = metadata_df.merge(corpus_df)
metadata_df.head()

## 5. Ergebnis speichern  

In [ ]:
# 🚀 Create result directory path
result_path = Path(r"../results")
if not result_path.exists():
    result_path.mkdir()

In [ ]:
result_path = Path(r"../results")
metadata_df.to_csv(result_path / "metadata_with_readability_scores.csv", index=False)